In [ ]:
# Теперь нужно решить, удалять ли строчки, которые df.groupby('major_cat_name')['major_cat_name'].count()

# Очистка данных: Ticket Sales Data
Набор данных выглядит совсем не очень. В нём есть пропущенные значения, избыточная, и не предназначенная для анализа информация. Ваша задача Привести этот набор данных в приличный вид. Сделайте так, чтобы на этот набор данных было приятно смотреть, и было легко строить на основе него дальнейшую аналитику.


## Шаг 1. Импортируем нужные библиотеки


In [ ]:
# ваш код тут
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from google.colab import files

## Шаг 2. Загрузаем данные в переменную df
Файл расположен по адресу 'https://drive.google.com/open?id=1KIsMPuZ3tWuCOM-LNJU6c29HV6HR4gwO'
И загружен в проект как sales.csv

---



In [ ]:
# ваш код тут
import io
df = pd.read_csv(io.BytesIO(uploaded['sales.csv']), encoding='latin1')

## Шаг 3. Первоначальный осмотр

In [ ]:
# Берем кусочек датафрейма и визуально оцениваем данные, что у нас имеются
# 4 случайные записи из датафрейма
df.sample(4)

,Unnamed: 0,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,ticket_text,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,venue_postal_cd_sgmt_1,sales_platform_cd,print_flg,la_valid_tkt_event_flg,fin_mkt_nm,web_session_cookie_val,gndr_cd,age_yr,income_amt,edu_val,edu_1st_indv_val,edu_2nd_indv_val,adults_in_hh_num,married_ind,child_present_ind,home_owner_ind,occpn_val,occpn_1st_val,occpn_2nd_val,dist_to_ven
1454,1455,698c10bfb9fc38778c57,b9fcab6d9e4154e2a3c6,b85143bf51323b72e53c,bb0da216f48efac4902d,Janet Jackson: Unbreakable World Tour,Janet Jackson,NaN,CONCERTS,ROCK/POP,CONCERTS,Janet Jackson: Unbreakable World Tour,LIVE NATION PRESENTS JANET JACKSON ...,1,127.0,eTicket,2015-09-24 00:00:00,2015-09-23,2015-06-15,2015-06-22,2015-06-21 18:31:31,2015-06-21,2015-09-04,EST,ORLANDO,FLORIDA,32801,mobile.ticketmaster.us,T,Y,Orlando,bb0da216f48efac4902d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,2008,99afec393b1d8a6726d1,d0b2f8ad63a0301c1d78,b85143bf51323b72e53c,61087f23042cc959afc5,Gexa Energy Pavilion Vip Parking: Iron Maiden,Hertz Vip Parking,NaN,MISC,PARKING,PARKING,Gexa Energy Pavilion Vip Parking: Iron Maiden,THIS TICKET IS VALID FOR PARKING ONL...,1,20.0,TicketFast,2012-08-18 00:30:00,2012-08-17,NaN,2012-03-03,2012-03-06 15:01:28,2012-03-06,2012-08-01,CST,DALLAS,TEXAS,75210,www.livenation.com,T,N,Dallas,61087f23042cc959afc5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1748,1749,452c0207617cbccf331b,6cdc2e270775b7e2f709,b85143bf51323b72e53c,c3dde72a2dd2f142d91c,Fast Lane Voucher: America's Most Wanted,Vip Fast Lane,NaN,MISC,SPECIAL ENTRY (UPSELL),UPSELL,Fast Lane Voucher: America's Most Wanted,THIS VOUCHER IS VALID FOR FAST LANE ACCES...,2,10.0,TicketFast,2013-07-12 23:00:00,2013-07-12,NaN,NaN,2013-04-19 15:27:09,2013-04-19,NaN,EST,ATLANTA,GEORGIA,30315,www.concerts.livenation.com,F,N,Atlanta,c3dde72a2dd2f142d91c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1938,1939,330dd8dd18a1ca921530,9f5a2f3c272fa7250880,e9c061d9b3d4341c5287,4643cf62f5f6ef31d788,Ricky Martin: One World Tour 2015,Ricky Martin,Ha*Ash,CONCERTS,LATIN MUSIC,CONCERTS,Ricky Martin: One World Tour 2015,LIVE NATION PRESENTS RICKY MARTIN ...,4,488.0,TicketFast,2015-09-30 00:00:00,2015-09-29,2015-04-01,2015-04-03,2015-05-11 02:34:50,2015-05-10,2015-05-10,CST,HIDALGO,TEXAS,78557,www.ticketmaster.com,T,Y,San Antonio,4643cf62f5f6ef31d788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Это данные о продаже билетов на концерты, есть информация:
1. Название концерта
2. Тип концерта
3. Жанр концерта
4. Дата покупки билета
5. Тип покупки клиента (электронные/ в кассе) 
6. Количество купленных билетов
7. Место покупки
8. Место проведение концерта
9. Информация о покупателе (пол, возраст)

Что хочется узнать из данных, после очистки:
1. Сколько билетов продано на каждый концерт
2. Какой наиболее распространенный тип покупки билетов
3. Какой наиболее популярный жанр концерта
4. Сколько в среднем продается билетов за одну покупку
5. Какой наиболее популярный сайт для покупки билетов


In [ ]:
# размер нашего датафрейма 
# 5000 строк и 46 столбцов
df.shape 

(5000, 46)

In [ ]:
# смотрим, какие столбцы есть:
# всего 46 столбцов, из них
# 2 - целочисленные
# 8 - числа c плавающей точкой
# 36 - строки
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 46 columns):
Unnamed: 0                5000 non-null int64
event_id                  5000 non-null object
primary_act_id            5000 non-null object
secondary_act_id          5000 non-null object
purch_party_lkup_id       5000 non-null object
event_name                5000 non-null object
primary_act_name          5000 non-null object
secondary_act_name        1586 non-null object
major_cat_name            5000 non-null object
minor_cat_name            5000 non-null object
la_event_type_cat         5000 non-null object
event_disp_name           5000 non-null object
ticket_text               5000 non-null object
tickets_purchased_qty     5000 non-null int64
trans_face_val_amt        5000 non-null float64
delivery_type_cd          5000 non-null object
event_date_time           5000 non-null object
event_dt                  5000 non-null object
presale_dt                2108 non-null object
o

In [ ]:
# Смотрим агрегированные значения по числовым столбцам
df.describe()

,Unnamed: 0,tickets_purchased_qty,trans_face_val_amt,age_yr,income_amt,adults_in_hh_num,married_ind,child_present_ind,home_owner_ind,dist_to_ven
count,5000.000000,5000.00000,5000.000000,168.000000,154.000000,176.000000,157.000000,130.000000,138.000000,323.000000
mean,2500.500000,1.63900,77.077446,44.369048,86396.103896,2.579545,0.668790,0.561538,0.942029,158.188854
std,1443.520003,1.11361,123.864788,16.531067,58895.045547,1.490707,0.472155,0.498118,0.234540,395.678376
min,1.000000,1.00000,1.000000,18.000000,10000.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1250.750000,1.00000,20.000000,30.000000,45000.000000,1.000000,0.000000,0.000000,1.000000,12.000000
50%,2500.500000,1.00000,30.000000,44.000000,62500.000000,2.000000,1.000000,1.000000,1.000000,26.000000
75%,3750.250000,2.00000,85.000000,56.000000,87500.000000,4.000000,1.000000,1.000000,1.000000,77.500000
max,5000.000000,8.00000,1520.880000,94.000000,200000.000000,6.000000,1.000000,1.000000,1.000000,2548.000000


##Шаг 4. Удаляем лишние данные


###Проблема 1: Неправильные индексы
Есть отдельный столбцец с индексом  "Unnamed: 0", дублирующий столбец с индексом
Переименуем индекс и удалим дублирующий столбцец "Unnamed: 0"


In [ ]:
# Сначала переименуем столбец индексов 
df.index.name = 'id'
# Удалим столбец "Unnamed: 0"
df.drop(columns=["Unnamed: 0"], inplace=True)
df.sample(4)

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,ticket_text,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,venue_postal_cd_sgmt_1,sales_platform_cd,print_flg,la_valid_tkt_event_flg,fin_mkt_nm,web_session_cookie_val,gndr_cd,age_yr,income_amt,edu_val,edu_1st_indv_val,edu_2nd_indv_val,adults_in_hh_num,married_ind,child_present_ind,home_owner_ind,occpn_val,occpn_1st_val,occpn_2nd_val,dist_to_ven
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4749,711026f34959b2d3e8c6,1a3e9aecd0617706a794,f53529c5679ea6ca5a48,dbf6a024be268a6e3fc0,Overnight Camping For Dave Matthews Band,Gorge Camping,Dave Matthews Band,MISC,CAMPING,INVALID,Overnight Camping For Dave Matthews Band,%OVERNIGHT C A M P I N G%* GOOD THIS DAY ONLY ...,1,40.0,Mail,2006-09-04 02:00:00,2006-09-03,2006-04-06,2006-04-08,2006-06-26 00:00:00,2006-06-26,2006-06-28,PST,QUINCY,WASHINGTON,98848,NaN,T,N,Seattle,dbf6a024be268a6e3fc0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2671,9e4803ca925cca4760c6,0e7886b245dfe1b7f6e0,ff101e2436655cde2264,c92084f0505bb134f51b,92.3 Wmxd Holiday Jam 2015,Jill Scott,New Edition,CONCERTS,R & B,CONCERTS,92.3 Wmxd Holiday Jam 2015,MIX 92.3 PRESENTS HOLIDAY JAM 201...,2,179.9,eTicket,2015-12-13 00:00:00,2015-12-12,2015-10-15,2015-10-16,2015-11-05 17:38:13,2015-11-05,2015-12-12,EST,DETROIT,MICHIGAN,48226,iphone.ticketmaster.us,T,Y,Michigan,c92084f0505bb134f51b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,0c6aa3194f5e86ae1368,ac4b847b3fde66f2117e,b85143bf51323b72e53c,60add2406741cd4fa5fc,Fast Lane/skip The Line Voucher,Fast Lane Access,NaN,MISC,SPECIAL ENTRY (UPSELL),UPSELL,Fast Lane/skip The Line Voucher,1 BOTTLED WATER PER TICKET FAST LANE-SKIP THE ...,3,30.0,Mail,2011-12-04 00:00:00,2011-12-03,NaN,2011-10-15,2011-11-13 15:46:37,2011-11-13,2011-11-18,EST,DETROIT,MICHIGAN,48201,NaN,T,N,Michigan,60add2406741cd4fa5fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1228,573b33137b63047012a6,dddbb3e176cd226421c6,8a478f02a60569cf3bdf,ae0f5868b9cef5e31452,Jason Aldean: 2015 burn it down tour,Jason Aldean,Cole Swindell,CONCERTS,COUNTRY,CONCERTS,Jason Aldean: 2015 burn it down tour,WWW.JASONALDEAN.COM JASON ALDEAN ...,2,151.0,eTicket,2015-09-17 23:30:00,2015-09-17,2015-05-06,2015-05-08,2015-05-08 14:07:34,2015-05-08,2015-09-17,EST,TORONTO,ONTARIO,M6K,www.ticketmaster.ca,T,Y,Toronto,ae0f5868b9cef5e31452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Теперь удаляем дубликаты, т.к раньше первый столбец был уникален и дубликатов не могло быть
df.drop_duplicates(inplace=True)

### Проблема 2: Недостаточно данных
Удалим 14 столбцов, где меньше 350 не нулевых значений
Цифра 350 выбрана на основе функции df.info(), где указано количество ненулевых значений


In [ ]:
# из данных полученных через df.info выделяем следующие 14 столбцов
#gndr_cd                   
#age_yr                    
#income_amt              
#edu_val                
#edu_1st_indv_val          
#edu_2nd_indv_val         
#adults_in_hh_num          
#married_ind               
#child_present_ind         
#home_owner_ind            
#occpn_val                 
#occpn_1st_val            
#occpn_2nd_val            
#dist_to_ven
columns_to_drop = ['gndr_cd', 'age_yr', 'income_amt', 'income_amt', 'edu_val', 'edu_1st_indv_val', 'edu_2nd_indv_val',\
'adults_in_hh_num', 'married_ind', 'child_present_ind', 'home_owner_ind', 'occpn_val', 'occpn_1st_val', 'occpn_2nd_val', 'dist_to_ven']
df.drop(columns_to_drop, axis=1, inplace= True)      

In [ ]:
df.head(4)       

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,ticket_text,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,venue_postal_cd_sgmt_1,sales_platform_cd,print_flg,la_valid_tkt_event_flg,fin_mkt_nm,web_session_cookie_val
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,abcaf1adb99a935fc661,43f0436b905bfa7c2eec,b85143bf51323b72e53c,7dfa56dd7d5956b17587,Xfinity Center Mansfield Premier Parking: Flor...,XFINITY Center Mansfield Premier Parking,NaN,MISC,PARKING,PARKING,Xfinity Center Mansfield Premier Parking: Flor...,THIS TICKET IS VALID FOR PARKING ONL...,1,45.0,eTicket,2015-09-12 23:30:00,2015-09-12,NaN,2015-05-15,2015-09-11 18:17:45,2015-09-11,2015-09-12,EST,MANSFIELD,MASSACHUSETTS,02048,www.concerts.livenation.com,T,N,Boston,7dfa56dd7d5956b17587
1,6c56d7f08c95f2aa453c,1a3e9aecd0617706a794,f53529c5679ea6ca5a48,4f9e6fc637eaf7b736c2,Gorge Camping - dave matthews band - sept 3-7,Gorge Camping,Dave Matthews Band,MISC,CAMPING,INVALID,Gorge Camping - dave matthews band - sept 3-7,%OVERNIGHT C A M P I N G%* * * * * *%GORGE CAM...,1,75.0,TicketFast,2009-09-05 01:00:00,2009-09-04,NaN,2009-03-13,2009-07-06 00:00:00,2009-07-05,2009-09-01,PST,QUINCY,WASHINGTON,98848,NaN,T,N,Seattle,4f9e6fc637eaf7b736c2
2,c7ab4524a121f9d687d2,4b677c3f5bec71eec8d1,b85143bf51323b72e53c,6c2545703bd527a7144d,Dodge Theatre Adams Street Parking - benise,Parking Event,NaN,MISC,PARKING,PARKING,Dodge Theatre Adams Street Parking - benise,ADAMS STREET GARAGE%PARKING FOR 4/21/06 ONLY%D...,1,5.0,TicketFast,2006-04-22 01:30:00,2006-04-21,NaN,2006-02-25,2006-04-05 00:00:00,2006-04-05,2006-04-05,MST,PHOENIX,ARIZONA,85003,NaN,T,N,Arizona,6c2545703bd527a7144d
3,394cb493f893be9b9ed1,b1ccea01ad6ef8522796,b85143bf51323b72e53c,527d6b1eaffc69ddd882,Gexa Energy Pavilion Vip Parking : kid rock wi...,Gexa Energy Pavilion VIP Parking,NaN,MISC,PARKING,PARKING,Gexa Energy Pavilion Vip Parking : kid rock wi...,THIS TICKET IS VALID FOR PARKING ONL...,1,20.0,Mail,2011-09-03 00:00:00,2011-09-02,NaN,2011-04-22,2011-07-01 17:38:50,2011-07-01,2011-07-06,CST,DALLAS,TEXAS,75210,NaN,T,N,Dallas,527d6b1eaffc69ddd882


###Проблема 3: Данные о покупателях билетов, а не о концертах
Мы анализируем продажу билетов в общем, а не покупателей билетов.
Поэтому нужно удалить те столбцы, в которых есть информация о покупателях - например почтовый индекс и те столбцы, которые не будут полезны при анализе. 
А также столбцы, в которых я не смогла понять, какая информация находится

In [ ]:
# Выделила данные, которые сложно проанализировать:
# fin_mkt_nm - город покупателя
# venue_postal_cd_sgmt_1	- почтовый индекс проведения концерта
# ticket_text - текст на билете

columns_to_drop_2 = ['fin_mkt_nm', 'venue_postal_cd_sgmt_1', 'ticket_text']
df.drop(columns_to_drop_2, axis=1, inplace= True)     

In [ ]:
df.head(4) 

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,sales_platform_cd,print_flg,la_valid_tkt_event_flg,web_session_cookie_val
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,abcaf1adb99a935fc661,43f0436b905bfa7c2eec,b85143bf51323b72e53c,7dfa56dd7d5956b17587,Xfinity Center Mansfield Premier Parking: Flor...,XFINITY Center Mansfield Premier Parking,NaN,MISC,PARKING,PARKING,Xfinity Center Mansfield Premier Parking: Flor...,1,45.0,eTicket,2015-09-12 23:30:00,2015-09-12,NaN,2015-05-15,2015-09-11 18:17:45,2015-09-11,2015-09-12,EST,MANSFIELD,MASSACHUSETTS,www.concerts.livenation.com,T,N,7dfa56dd7d5956b17587
1,6c56d7f08c95f2aa453c,1a3e9aecd0617706a794,f53529c5679ea6ca5a48,4f9e6fc637eaf7b736c2,Gorge Camping - dave matthews band - sept 3-7,Gorge Camping,Dave Matthews Band,MISC,CAMPING,INVALID,Gorge Camping - dave matthews band - sept 3-7,1,75.0,TicketFast,2009-09-05 01:00:00,2009-09-04,NaN,2009-03-13,2009-07-06 00:00:00,2009-07-05,2009-09-01,PST,QUINCY,WASHINGTON,NaN,T,N,4f9e6fc637eaf7b736c2
2,c7ab4524a121f9d687d2,4b677c3f5bec71eec8d1,b85143bf51323b72e53c,6c2545703bd527a7144d,Dodge Theatre Adams Street Parking - benise,Parking Event,NaN,MISC,PARKING,PARKING,Dodge Theatre Adams Street Parking - benise,1,5.0,TicketFast,2006-04-22 01:30:00,2006-04-21,NaN,2006-02-25,2006-04-05 00:00:00,2006-04-05,2006-04-05,MST,PHOENIX,ARIZONA,NaN,T,N,6c2545703bd527a7144d
3,394cb493f893be9b9ed1,b1ccea01ad6ef8522796,b85143bf51323b72e53c,527d6b1eaffc69ddd882,Gexa Energy Pavilion Vip Parking : kid rock wi...,Gexa Energy Pavilion VIP Parking,NaN,MISC,PARKING,PARKING,Gexa Energy Pavilion Vip Parking : kid rock wi...,1,20.0,Mail,2011-09-03 00:00:00,2011-09-02,NaN,2011-04-22,2011-07-01 17:38:50,2011-07-01,2011-07-06,CST,DALLAS,TEXAS,NaN,T,N,527d6b1eaffc69ddd882


##Шаг 5. Приводим данные к оптимальным типам

###Проблема 1 : Даты неправильного формата
В данных есть столбцы с датами, но все они типа object.
Их нужно преобразовать в datatime64

In [ ]:
#Визуально выделила следующие значения в которых записана дата:
#event_date_time
#event_dt
#presale_dt
#onsale_dt
#sales_ord_create_dttm
#sales_ord_tran_dt	
#print_dt

#Приведем их к формату datetime64
columns_to_datetime = ["event_date_time", "event_dt", "presale_dt", "onsale_dt", "sales_ord_create_dttm"\
    ,"sales_ord_tran_dt", "print_dt"]
df[columns_to_datetime] = df[columns_to_datetime].apply(pd.to_datetime)

###Проблема 2: Значения да/нет неправильного формата
В данных есть столбцы со значениями true/false, тип у них строчный обозначения Y/N. Нужео преобразовать в bool64, чтобы сэкономить место и удобнее применять функции pandas


In [ ]:
### Есть 2 один столбца с такими данными
# print_flg
# la_valid_tkt_event_flg
# сначала проверим, какие есть возможные значения в этих столбцах
df.print_flg.unique()        

array(['T ', 'F '], dtype=object)

In [ ]:
df.la_valid_tkt_event_flg.unique()     

array(['N ', 'Y '], dtype=object)

In [ ]:
# Заменим значения с помощью словаря с корректными значениями и функции replace
# Обращаем внимание, что символы T и F имеют пробел в конце, которые мы учитываем в словаре
bool1_map = {'T ': True, 
             'F ': False}
df['print_flg'] = df['print_flg'].replace(bool1_map)

bool2_map = {'Y ': True, 
             'N ': False}
df['la_valid_tkt_event_flg'] = df['la_valid_tkt_event_flg'].replace(bool2_map)

In [ ]:
df

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,sales_platform_cd,print_flg,la_valid_tkt_event_flg,web_session_cookie_val
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,abcaf1adb99a935fc661,43f0436b905bfa7c2eec,b85143bf51323b72e53c,7dfa56dd7d5956b17587,Xfinity Center Mansfield Premier Parking: Flor...,XFINITY Center Mansfield Premier Parking,NaN,MISC,PARKING,PARKING,Xfinity Center Mansfield Premier Parking: Flor...,1,45.0,eTicket,2015-09-12 23:30:00,2015-09-12,NaT,2015-05-15,2015-09-11 18:17:45,2015-09-11,2015-09-12,EST,MANSFIELD,MASSACHUSETTS,www.concerts.livenation.com,True,False,7dfa56dd7d5956b17587
1,6c56d7f08c95f2aa453c,1a3e9aecd0617706a794,f53529c5679ea6ca5a48,4f9e6fc637eaf7b736c2,Gorge Camping - dave matthews band - sept 3-7,Gorge Camping,Dave Matthews Band,MISC,CAMPING,INVALID,Gorge Camping - dave matthews band - sept 3-7,1,75.0,TicketFast,2009-09-05 01:00:00,2009-09-04,NaT,2009-03-13,2009-07-06 00:00:00,2009-07-05,2009-09-01,PST,QUINCY,WASHINGTON,NaN,True,False,4f9e6fc637eaf7b736c2
2,c7ab4524a121f9d687d2,4b677c3f5bec71eec8d1,b85143bf51323b72e53c,6c2545703bd527a7144d,Dodge Theatre Adams Street Parking - benise,Parking Event,NaN,MISC,PARKING,PARKING,Dodge Theatre Adams Street Parking - benise,1,5.0,TicketFast,2006-04-22 01:30:00,2006-04-21,NaT,2006-02-25,2006-04-05 00:00:00,2006-04-05,2006-04-05,MST,PHOENIX,ARIZONA,NaN,True,False,6c2545703bd527a7144d
3,394cb493f893be9b9ed1,b1ccea01ad6ef8522796,b85143bf51323b72e53c,527d6b1eaffc69ddd882,Gexa Energy Pavilion Vip Parking : kid rock wi...,Gexa Energy Pavilion VIP Parking,NaN,MISC,PARKING,PARKING,Gexa Energy Pavilion Vip Parking : kid rock wi...,1,20.0,Mail,2011-09-03 00:00:00,2011-09-02,NaT,2011-04-22,2011-07-01 17:38:50,2011-07-01,2011-07-06,CST,DALLAS,TEXAS,NaN,True,False,527d6b1eaffc69ddd882
4,55b5f67e618557929f48,91c03a34b562436efa3c,b85143bf51323b72e53c,8bd62c394a35213bdf52,Premier Parking - motley crue,White River Amphitheatre Premier Parking,NaN,MISC,PARKING,PARKING,Premier Parking - motley crue,1,20.0,Mail,2005-07-31 01:00:00,2005-07-30,2005-03-02,2005-03-04,2005-06-18 00:00:00,2005-06-18,2005-06-28,PST,AUBURN,WASHINGTON,NaN,True,False,8bd62c394a35213bdf52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,79372bc1e7e14dfd3f82,a14232befff04be1e2f3,b85143bf51323b72e53c,8a03312852bd2a785f2c,Premier Parking - eric clapton - pnc bank arts...,PNC Bank Arts Center Holmdel Premier Parking,NaN,MISC,PARKING,PARKING,Premier Parking - eric clapton - pnc bank arts...,1,30.0,Mail,2008-05-22 23:30:00,2008-05-22,NaT,2008-03-10,2008-03-10 00:00:00,2008-03-10,2008-03-15,EST,HOLMDEL,NEW JERSEY,NaN,True,False,8a03312852bd2a785f2c
4996,f3b0de6a19e2a16e4fb1,83e5715f0ed3e13e6398,b2b558b1fe3efad19edb,f032fe520e8313bef880,Sublime With Rome / rebelution,Sublime with Rome,Rebelution,CONCERTS,ROCK/POP,CONCERTS,Sublime With Rome / rebelution,2,26.0,eTicket,2015-07-17 05:00:00,2015-07-16,2015-04-14,2015-04-20,2015-07-14 16:02:00,2015-07-14,2015-07-15,PST,CHULA VISTA,CALIFORNIA,www.concerts.livenation.com,True,True,f032fe520e8313bef880
4997,7c9e2f6847a2dbe4ebff,9ea8ab40f58d18305ac8,b85143bf51323b72e53c,f270f4daa5c3dd61a8c5,Premier Parking : journey with heart and cheap...,Sleep Train Amphitheatre Premier Parking,NaN,MISC,PARKING,PARKING,Premier Parking : journey with heart and cheap...,1,20.0,Mail,2008-09-27 02:00:00,2008-09-26,NaT,2008-05-17,2008-05-18 00:00:00,2008-05-27,2008-05-29,PST,WHEATLAND,CALIFORNIA,NaN,True,False,f270f4daa5c3dd61a8c5


### Проблема 3: маленькие числа занимают много памяти
Есть два столбца с числами

tickets_purchased_qty - Количество билетов

trans_face_val_amt - стоимость билета 
Они типов int64 и float64 соответственно

На основе df.describe изменим формат данных, без потери точности

In [ ]:
# Почему - то тип указывается отличный от функции info()
# Но тут мы смотрим агрегированные данные:
df.tickets_purchased_qty.describe()

count    5000.00000
mean        1.63900
std         1.11361
min         1.00000
25%         1.00000
50%         1.00000
75%         2.00000
max         8.00000
Name: tickets_purchased_qty, dtype: float64

In [ ]:
# Для этих данных вполне достаточно формата uint8 числа от 0 до 255
df['tickets_purchased_qty'] = pd.to_numeric(df['tickets_purchased_qty'], downcast='unsigned')

In [ ]:
# Аналогично для trans_face_val_amt
df.trans_face_val_amt.describe()

count    5000.000000
mean       77.077446
std       123.864788
min         1.000000
25%        20.000000
50%        30.000000
75%        85.000000
max      1520.880000
Name: trans_face_val_amt, dtype: float64

In [ ]:
df['trans_face_val_amt'] = pd.to_numeric(df['trans_face_val_amt'], downcast='float')

In [ ]:
# Снова смотрим на info
# данные изменились на uint8 и float32
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 28 columns):
event_id                  5000 non-null object
primary_act_id            5000 non-null object
secondary_act_id          5000 non-null object
purch_party_lkup_id       5000 non-null object
event_name                5000 non-null object
primary_act_name          5000 non-null object
secondary_act_name        1586 non-null object
major_cat_name            5000 non-null object
minor_cat_name            5000 non-null object
la_event_type_cat         5000 non-null object
event_disp_name           5000 non-null object
tickets_purchased_qty     5000 non-null uint8
trans_face_val_amt        5000 non-null float32
delivery_type_cd          5000 non-null object
event_date_time           5000 non-null datetime64[ns]
event_dt                  5000 non-null datetime64[ns]
presale_dt                2108 non-null datetime64[ns]
onsale_dt                 4899 non-null datetime64[ns]
sales_ord_creat

##Шаг 6. Форматируем и оптимизируем данные
После крупной чистки данных, смотрим на каждый столбец и думаем, можем ли мы что-то улучшить

###Идея 1: Разделить столбец с данными о сайте покупки


In [ ]:
# Смотрим на значения столбца sales_platform_cd -сайт где покупали билеты
df1 = df.sales_platform_cd
df1.unique()

array(['www.concerts.livenation.com', nan, 'www.livenation.com',
       'mobile.livenation.us', 'www.ticketmaster.com',
       'iphone.ticketmaster.us', 'android.ticketmaster.us',
       'iphone2.livenation.us', 'mobile.ticketmaster.us',
       'www.ticketmaster.ca', 'mobile.ticketmaster.ca',
       'androidphone2.livenation.us', 'iphone.ticketmaster.ca',
       'www.ticketmaster.com.mx'], dtype=object)

In [ ]:
# Этот столбец можно разделить на 2:
# 1. Cайт, где куплен билет, их всего 2: ticketmaster.com и livenation.com
# 2. Устройство с помощью которого покупатель заходил - iphone, android, mobile (когда неизвестна платформа телефона), desktop (с пк или ноутбука)

site_map = {}
for key in ['www.concerts.livenation.com', 'www.livenation.com', 'mobile.livenation.us','mobile.livenation.us', 'iphone2.livenation.us', 'androidphone2.livenation.us']:
    site_map[key] = 'www.livenation.com'
for key in ['www.ticketmaster.com', 'iphone.ticketmaster.us', 'android.ticketmaster.us', 'mobile.ticketmaster.us', 'www.ticketmaster.ca', 'iphone.ticketmaster.ca', 'www.ticketmaster.com.mx']:
    site_map[key] = 'www.ticketmaster.com'

device_map = {} 
for key in ['iphone.ticketmaster.us', 'iphone2.livenation.us', 'iphone.ticketmaster.ca']:
    device_map[key] = 'iphone'
for key in ['android.ticketmaster.us']:
    device_map[key] = 'android'
for key in ['mobile.livenation.us', 'mobile.ticketmaster.us', 'mobile.ticketmaster.ca', 'androidphone2.livenation.us']:
    device_map[key] = 'mobile'
for key in ['www.concerts.livenation.com','www.livenation.com',  'www.ticketmaster.com','www.ticketmaster.ca','www.ticketmaster.com.mx']:
    device_map[key] = 'desktop'

df['Sales site'] = df['sales_platform_cd'].map(site_map)
df['Sales device'] = df['sales_platform_cd'].map(device_map)

# Удаляем колонку sales_platform_cd
df.drop('sales_platform_cd', axis=1, inplace= True)  

###Идея 2: Убрать столбец с типом мероприятия

In [ ]:
# Посмотрим, какие значения есть в столбце major_cat_name - основная категория мероприятия
df.major_cat_name.unique()

array(['MISC', 'CONCERTS', 'ARTS', 'SPORTS', 'FAMILY'], dtype=object)

In [ ]:
# Видим 5 категорий, есть догадка, что категорий не "CONCERT" или "MISC" (Разное) - очень мало
# Проверим: 
df.groupby('major_cat_name')['major_cat_name'].count()

major_cat_name
ARTS          25
CONCERTS    1998
FAMILY         4
MISC        2972
SPORTS         1
Name: major_cat_name, dtype: int64

In [ ]:
# Таких строчек всего 30, как мы и думали
# Посмотрим на эти строки:
df[(df['major_cat_name'] != 'MISC')& (df['major_cat_name'] != 'CONCERTS')]

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,print_flg,la_valid_tkt_event_flg,web_session_cookie_val,Sales site,Sales device
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
289,62c32c8476c7a5547cc7,77395241bdc49e9146e1,b85143bf51323b72e53c,f680ad98ca6be7336ca4,The Tenors: Under One Sky Tour,The Tenors,NaN,ARTS,CLASSICAL/VOCAL,ARTS,The Tenors: Under One Sky Tour,1,45.000000,eTicket,2015-09-29 01:30:00,2015-09-28,2015-05-04,2015-05-08,2015-09-15 15:51:23,2015-09-15,2015-09-22,PST,CALGARY,ALBERTA,True,True,f680ad98ca6be7336ca4,www.ticketmaster.com,desktop
1215,01de4515af60ffac1529,77395241bdc49e9146e1,b85143bf51323b72e53c,e62ffca8667b54ca5ea1,The Tenors: Under One Sky Tour,The Tenors,NaN,ARTS,CLASSICAL/VOCAL,ARTS,The Tenors: Under One Sky Tour,3,280.109985,eTicket,2015-11-25 00:30:00,2015-11-24,2015-05-04,2015-05-08,2015-05-08 21:29:07,2015-05-08,2015-05-08,EST,TORONTO,ONTARIO,True,True,e62ffca8667b54ca5ea1,NaN,mobile
1365,22eba5c543ec2b9f01b5,3849eb0c5852ab4bf27f,b85143bf51323b72e53c,2bcce175818b3d7ccf10,Live Nation Presents The Divas Of Drag,The Divas of Drag,NaN,ARTS,PERFORMANCE ART,ARTS,Live Nation Presents The Divas Of Drag,1,18.000000,eTicket,2016-03-27 00:00:00,2016-03-26,2016-02-03,2016-02-05,2016-02-21 18:16:27,2016-02-21,2016-02-21,EST,PHILADELPHIA,PENNSYLVANIA,True,True,2bcce175818b3d7ccf10,www.livenation.com,mobile
1660,17ede687a02b6412663c,ad873065e23f669111c0,b85143bf51323b72e53c,2ffd2824801d38be2f9e,Lord Of The Dance-dangerous Games,Lord of the Dance - Dangerous Games,NaN,ARTS,BALLET/DANCE,ARTS,Lord Of The Dance-dangerous Games,2,138.000000,eTicket,2016-02-21 23:00:00,2016-02-21,2015-12-03,2015-12-07,2016-02-18 00:38:45,2016-02-17,2016-02-18,EST,ATLANTA,GEORGIA,True,True,2ffd2824801d38be2f9e,www.ticketmaster.com,desktop
1870,89d5964a926b01e553b2,60f4fc544c7f8b2bf5ce,b85143bf51323b72e53c,e739530177c4f39faf94,Chippendales: The 2016 break the rules tour,Chippendales,NaN,ARTS,OTHER,ARTS,Chippendales: The 2016 break the rules tour,1,35.000000,eTicket,2016-02-11 00:30:00,2016-02-10,2015-09-23,2015-09-25,2016-02-09 02:55:29,2016-02-08,2016-02-09,EST,BOSTON,MASSACHUSETTS,True,True,e739530177c4f39faf94,www.livenation.com,desktop
2045,4e4e62b0c16fd498f993,c30d53db9fc34fa0a95d,b85143bf51323b72e53c,326f7f2065168c09806a,The Next Step Wild Rhythm Tour,The Next Step Dancers,NaN,ARTS,BALLET/DANCE,ARTS,The Next Step Wild Rhythm Tour,2,99.000000,ISPU,2016-02-20 00:30:00,2016-02-19,2015-10-16,2015-10-16,2015-12-19 21:47:21,2015-12-19,2016-02-19,PST,WINNIPEG,MANITOBA,True,True,326f7f2065168c09806a,www.ticketmaster.com,desktop
2241,ecc2eb62291cb68cec74,e22304c809021c744436,b85143bf51323b72e53c,9d24f287d7e9e1a6dbfd,Magic Men LIVE!,Magic Men LIVE!,NaN,ARTS,OTHER,ARTS,Magic Men LIVE!,4,101.000000,eTicket,2016-02-24 01:00:00,2016-02-23,NaT,2016-02-01,2016-02-17 20:14:28,2016-02-17,2016-02-23,CST,SAN ANTONIO,TEXAS,True,True,9d24f287d7e9e1a6dbfd,www.livenation.com,desktop
2244,a138ff85cd711787015a,58147a12f5a5fd913506,b85143bf51323b72e53c,21f89c9bf36e07464938,Wwe Presents Nxt Live,WWE,NaN,SPORTS,WRESTLING,SPORTS,Wwe Presents Nxt Live,2,150.000000,eTicket,2016-02-07 00:30:00,2016-02-06,2016-01-07,2016-01-08,2016-01-07 15:40:15,2016-01-07,2016-01-07,CST,INDIANAPOLIS,INDIANA,True,True,21f89c9bf36e07464938,www.livenation.com,desktop
2293,3b8e5f27b2fe01bac77a,201aebef1c6b5980be7d,b85143bf51323b72e53c,ead732f6fdced9e20a3d,The Wiggles,The Wiggles,NaN,FAMILY,CHILDREN'S THEATRE,FAMILY,The Wiggles,3,114.000000,eTicket,2015-09-26 20:00:00,2015-09-26,2015-05-19,2015-05-21,2015-05-26 04:04:51,2015-05-25,2015-09-23,EST,WASHINGTON,DISTRICT OF COLUMBIA,True,True,ead732f6fdced9e20a3d,www.livenation.com,mobile


In [ ]:
# Также заметила, что столбцы major_cat_name и la_event_type_cat очень похожи

df[(df['major_cat_name'] != df['la_event_type_cat']) & ((df['major_cat_name'] != "MISC") & (df['major_cat_name'] != "CONCERTS"))][['major_cat_name', 'la_event_type_cat']]

,major_cat_name,la_event_type_cat
id,,


In [ ]:
# Итого, после анализа решено удалить столбец major_cat_name, т.к это всё концерты
# А более подробная информация содержится в minor_cat_name
df.drop('major_cat_name', axis=1, inplace= True) 

In [ ]:
df

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,minor_cat_name,la_event_type_cat,event_disp_name,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,print_flg,la_valid_tkt_event_flg,web_session_cookie_val,Sales site,Sales device
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,abcaf1adb99a935fc661,43f0436b905bfa7c2eec,b85143bf51323b72e53c,7dfa56dd7d5956b17587,Xfinity Center Mansfield Premier Parking: Flor...,XFINITY Center Mansfield Premier Parking,NaN,PARKING,PARKING,Xfinity Center Mansfield Premier Parking: Flor...,1,45.0,eTicket,2015-09-12 23:30:00,2015-09-12,NaT,2015-05-15,2015-09-11 18:17:45,2015-09-11,2015-09-12,EST,MANSFIELD,MASSACHUSETTS,True,False,7dfa56dd7d5956b17587,www.livenation.com,desktop
1,6c56d7f08c95f2aa453c,1a3e9aecd0617706a794,f53529c5679ea6ca5a48,4f9e6fc637eaf7b736c2,Gorge Camping - dave matthews band - sept 3-7,Gorge Camping,Dave Matthews Band,CAMPING,INVALID,Gorge Camping - dave matthews band - sept 3-7,1,75.0,TicketFast,2009-09-05 01:00:00,2009-09-04,NaT,2009-03-13,2009-07-06 00:00:00,2009-07-05,2009-09-01,PST,QUINCY,WASHINGTON,True,False,4f9e6fc637eaf7b736c2,NaN,NaN
2,c7ab4524a121f9d687d2,4b677c3f5bec71eec8d1,b85143bf51323b72e53c,6c2545703bd527a7144d,Dodge Theatre Adams Street Parking - benise,Parking Event,NaN,PARKING,PARKING,Dodge Theatre Adams Street Parking - benise,1,5.0,TicketFast,2006-04-22 01:30:00,2006-04-21,NaT,2006-02-25,2006-04-05 00:00:00,2006-04-05,2006-04-05,MST,PHOENIX,ARIZONA,True,False,6c2545703bd527a7144d,NaN,NaN
3,394cb493f893be9b9ed1,b1ccea01ad6ef8522796,b85143bf51323b72e53c,527d6b1eaffc69ddd882,Gexa Energy Pavilion Vip Parking : kid rock wi...,Gexa Energy Pavilion VIP Parking,NaN,PARKING,PARKING,Gexa Energy Pavilion Vip Parking : kid rock wi...,1,20.0,Mail,2011-09-03 00:00:00,2011-09-02,NaT,2011-04-22,2011-07-01 17:38:50,2011-07-01,2011-07-06,CST,DALLAS,TEXAS,True,False,527d6b1eaffc69ddd882,NaN,NaN
4,55b5f67e618557929f48,91c03a34b562436efa3c,b85143bf51323b72e53c,8bd62c394a35213bdf52,Premier Parking - motley crue,White River Amphitheatre Premier Parking,NaN,PARKING,PARKING,Premier Parking - motley crue,1,20.0,Mail,2005-07-31 01:00:00,2005-07-30,2005-03-02,2005-03-04,2005-06-18 00:00:00,2005-06-18,2005-06-28,PST,AUBURN,WASHINGTON,True,False,8bd62c394a35213bdf52,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,79372bc1e7e14dfd3f82,a14232befff04be1e2f3,b85143bf51323b72e53c,8a03312852bd2a785f2c,Premier Parking - eric clapton - pnc bank arts...,PNC Bank Arts Center Holmdel Premier Parking,NaN,PARKING,PARKING,Premier Parking - eric clapton - pnc bank arts...,1,30.0,Mail,2008-05-22 23:30:00,2008-05-22,NaT,2008-03-10,2008-03-10 00:00:00,2008-03-10,2008-03-15,EST,HOLMDEL,NEW JERSEY,True,False,8a03312852bd2a785f2c,NaN,NaN
4996,f3b0de6a19e2a16e4fb1,83e5715f0ed3e13e6398,b2b558b1fe3efad19edb,f032fe520e8313bef880,Sublime With Rome / rebelution,Sublime with Rome,Rebelution,ROCK/POP,CONCERTS,Sublime With Rome / rebelution,2,26.0,eTicket,2015-07-17 05:00:00,2015-07-16,2015-04-14,2015-04-20,2015-07-14 16:02:00,2015-07-14,2015-07-15,PST,CHULA VISTA,CALIFORNIA,True,True,f032fe520e8313bef880,www.livenation.com,desktop
4997,7c9e2f6847a2dbe4ebff,9ea8ab40f58d18305ac8,b85143bf51323b72e53c,f270f4daa5c3dd61a8c5,Premier Parking : journey with heart and cheap...,Sleep Train Amphitheatre Premier Parking,NaN,PARKING,PARKING,Premier Parking : journey with heart and cheap...,1,20.0,Mail,2008-09-27 02:00:00,2008-09-26,NaT,2008-05-17,2008-05-18 00:00:00,2008-05-27,2008-05-29,PST,WHEATLAND,CALIFORNIA,True,False,f270f4daa5c3dd61a8c5,NaN,NaN


###Идея 3: Удалить один из дублирующихся столбцов с названием мероприятия

In [ ]:
# Заметила, что столбцы event_name и event_disp_name очень похожи
# Сравним значения в этих столбцах: 
df['event_name'].isin(df['event_disp_name']).value_counts()

True     4998
False       2
Name: event_name, dtype: int64

In [ ]:
# Всего 2 столбца из 5000 различаются 
# Посмотрим на них:
df[df['event_name'] != df['event_disp_name']][['event_name', 'event_disp_name']]

,event_name,event_disp_name
id,,
996,Soja,SOJA
4342,Ben Harper & the Innocent Criminals,Ben Harper & The Innocent Criminals


In [ ]:
# Они оказываются одинаковыми, с точностью до регистра 
# Поэтому удалим столбец event_disp_name
df.drop('event_disp_name', axis=1, inplace= True)  

### Идея 4: Убрать id и оставить только названия
Есть три пары столбцов

event_id event_name	
primary_act_id primary_act_name	
secondary_act_id secondary_act_name

Нужно избавиться от столбцов с id, при этом убедиться что все значения с одинаковым id имеют одинаковое название


In [ ]:
df.groupby('event_id')['event_name'].value_counts()

event_id              event_name                                                    
00071bfcbb27802045b2  Willie Nelson's 4th of july picnic presented by budweiser         1
001889c0940fa4fae413  Table Upgrade: Kyle Dunnigan - purchase one upgrade per person    1
00394b832432576b931a  Jon Pardi And Brothers Osborne                                    1
00454c7a335c30d982d7  The Rides                                                         1
004ac78a434a108b6421  Gorge Amphitheatre Premier Parking May 25                         1
                                                                                       ..
ffdb7f76c259bd015a54  Old National Centre Priority Pass: City And Colour                1
ffdccd4d093e3a0fbfe0  Paul Mccartney: Out There                                         1
ffdd948e2bc905c12faa  First Midwest Bank Amp. Parking: Crosby Stills Nash And Young     1
ffee9977209a80ad5997  John Fogerty-pnc Bank Arts Center Holmdel Premier Parking         1
fffafaef3af909b

In [ ]:
# Воспользуемся функцией transform, где аргумент first - заменять по первому вхождению
df['event_name'] = df.groupby('event_id')['event_name'].transform('first')

In [ ]:
df.groupby('event_id')['event_name'].value_counts()

event_id              event_name                                                    
00071bfcbb27802045b2  Willie Nelson's 4th of july picnic presented by budweiser         1
001889c0940fa4fae413  Table Upgrade: Kyle Dunnigan - purchase one upgrade per person    1
00394b832432576b931a  Jon Pardi And Brothers Osborne                                    1
00454c7a335c30d982d7  The Rides                                                         1
004ac78a434a108b6421  Gorge Amphitheatre Premier Parking May 25                         1
                                                                                       ..
ffdb7f76c259bd015a54  Old National Centre Priority Pass: City And Colour                1
ffdccd4d093e3a0fbfe0  Paul Mccartney: Out There                                         1
ffdd948e2bc905c12faa  First Midwest Bank Amp. Parking: Crosby Stills Nash And Young     1
ffee9977209a80ad5997  John Fogerty-pnc Bank Arts Center Holmdel Premier Parking         1
fffafaef3af909b

In [ ]:
# Как видим, уникальных значений стало меньше 3754 -> 3746
# Повторяем аналогичные действия для 2 остальных значений
df['primary_act_name'] = df.groupby('primary_act_id')['primary_act_name'].transform('first')
df['secondary_act_name'] = df.groupby('secondary_act_id')['secondary_act_name'].transform('first')

In [ ]:
# Теперь можно удалить столбцы с id 
columns_to_drop_3 = ['event_id', 'primary_act_id', 'secondary_act_id']
df.drop(columns_to_drop_3, axis=1, inplace= True)     

In [ ]:
df.sample(5)

,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,minor_cat_name,la_event_type_cat,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,print_flg,la_valid_tkt_event_flg,web_session_cookie_val,Sales site,Sales device
id,,,,,,,,,,,,,,,,,,,,,,,,
3125,9b01d0c5781c4759826b,Josh Groban,Josh Groban,Sarah McLachlan,ADULT CONTEMPORARY,CONCERTS,4,171.00,eTicket,2016-08-10 00:30:00,2016-08-09,2015-12-07,2015-12-11,2015-12-07 16:02:22,2015-12-07,2015-12-07,CST,CHICAGO,ILLINOIS,True,True,9b01d0c5781c4759826b,www.livenation.com,desktop
105,b6c3b2081938e74d7cde,Luke Bryan: Kill The Lights Tour 2016,Luke Bryan,Dustin Lynch,COUNTRY,CONCERTS,4,281.00,TicketFast,2016-03-09 02:30:00,2016-03-08,2015-12-02,2015-12-05,2015-12-05 17:06:25,2015-12-05,2015-12-05,CST,LAS CRUCES,NEW MEXICO,True,True,b6c3b2081938e74d7cde,www.ticketmaster.com,iphone
4994,5f0e8870d2d08f157042,Josh Groban,Josh Groban,NaN,ADULT CONTEMPORARY,CONCERTS,2,598.00,BWC,2015-09-27 00:00:00,2015-09-26,2015-04-20,2015-04-24,2015-04-20 15:00:07,2015-04-20,2015-08-26,EST,WALLINGFORD,CONNECTICUT,True,True,5f0e8870d2d08f157042,www.livenation.com,desktop
2492,23ebaf28caab80691bb2,Vip Parking: Honda Civic Tour,Hertz Vip Parking,NaN,PARKING,PARKING,1,25.00,TicketFast,2013-09-16 23:00:00,2013-09-16,NaT,2013-04-01,2013-04-11 17:57:52,2013-04-11,2013-09-14,EST,ATLANTA,GEORGIA,True,False,23ebaf28caab80691bb2,www.livenation.com,desktop
3140,31b20adfed6c9830a610,Bill Burr: 2015 south beach comedy festival,Bill Burr,South Beach Comedy Festival,COMEDY,ARTS,3,105.75,eTicket,2015-04-11 02:30:00,2015-04-10,NaT,2015-03-10,2015-04-10 00:00:11,2015-04-09,2015-04-10,EST,MIAMI BEACH,FLORIDA,True,True,31b20adfed6c9830a610,www.livenation.com,desktop


### Идея 5: Удалить один из дублирующихся столбцов с id покупки
Столбцы purch_party_lkup_id и web_session_cookie_val похожи
Возможно они дублируют друг друга и один можно удалить


In [ ]:
df2 = df[df['purch_party_lkup_id'] != df['web_session_cookie_val']]
df2

,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,minor_cat_name,la_event_type_cat,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,print_flg,la_valid_tkt_event_flg,web_session_cookie_val,Sales site,Sales device
id,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#Эти столбцы совпадают, удалим 'web_session_cookie_val'
df.drop('web_session_cookie_val', axis=1, inplace= True)    

### Идея 6: Разделить категорию мероприятия и тип билета
Есть два столбца

minor_cat_name - жанр концерта

la_event_type_cat	- тип билета

In [ ]:
# Посмотрим на то, какие данные в них лежат
df1 = df.minor_cat_name
df1.unique()

array(['PARKING', 'CAMPING', 'SPECIAL ENTRY (UPSELL)',
       'DANCE MUSIC/DANCE', 'COUNTRY', 'ROCK/POP',
       'CONCESSION VOUCHERS (UPSELL)', 'ALTERNATIVE ROCK',
       'DINNER PACKAGES (UPSELL)', 'COMEDY', 'RAP/URBAN', 'R & B',
       'OTHER', 'LATIN MUSIC', 'JAZZ', 'POP', 'CLUB ACCESS',
       'ADULT CONTEMPORARY', 'CHANSON FRANCAISE', 'HEAVY METAL',
       'CLASSICAL/VOCAL', 'ETHNIC/FOREIGN', 'PUNK', 'REGGAE', 'FESTIVALS',
       'TRIBUTE BAND', 'FOLK', 'BUS/TRANSPORTATION', 'PERFORMANCE ART',
       'BALLET/DANCE', 'SOUL', 'GOSPEL', 'WRESTLING',
       "CHILDREN'S THEATRE", 'THEATRE (MUSICAL)', 'HOLIDAY ATTRACTION',
       'LECTURE/SEMINAR', 'THEATRE (DRAMA)', 'BLUES', 'OLDIES',
       'CLASSICAL/SYMPHONIC', 'THEATRE OTHER', 'BLUEGRASS', 'MAGIC'],
      dtype=object)

In [ ]:
df1 = df.la_event_type_cat
df1.unique()

array(['PARKING', 'INVALID', 'UPSELL', 'CONCERTS', 'ARTS', 'SPORTS',
       'FAMILY'], dtype=object)

In [ ]:
# При рассмотрении этих данных заметила,
# Что флаг 'la_valid_tkt_event_flg' отражает то, является ли эта покупка билетом на концерт
df.groupby(['la_valid_tkt_event_flg', 'la_event_type_cat'])['la_valid_tkt_event_flg'].agg('count')

la_valid_tkt_event_flg  la_event_type_cat
False                   INVALID               171
                        PARKING              2324
                        UPSELL                490
True                    ARTS                  104
                        CONCERTS             1906
                        FAMILY                  4
                        SPORTS                  1
Name: la_valid_tkt_event_flg, dtype: int64

In [ ]:
# Выборка показала, что действительно значение True есть только для категорий ARTS, CONCERTS, FAMILY, SPORT 
# Сделаем аналогичную выборку для minor_cat_name
df.groupby(['la_valid_tkt_event_flg', 'minor_cat_name'])['la_valid_tkt_event_flg'].agg('count')

la_valid_tkt_event_flg  minor_cat_name              
False                   ALTERNATIVE ROCK                   1
                        BUS/TRANSPORTATION                 1
                        CAMPING                          158
                        CLUB ACCESS                       38
                        COMEDY                             1
                        CONCESSION VOUCHERS (UPSELL)     123
                        DINNER PACKAGES (UPSELL)          17
                        HOLIDAY ATTRACTION                 1
                        LATIN MUSIC                        1
                        LECTURE/SEMINAR                    1
                        OTHER                              8
                        PARKING                         2314
                        RAP/URBAN                          2
                        ROCK/POP                           8
                        SPECIAL ENTRY (UPSELL)           311
True                    ADULT CO

In [ ]:
# Для minor_cat_name есть строки с признаком False, где указаны жанры музыки, а не информация о типе билета
# Но как быстрое решение нужно удалить все строки, где 'la_valid_tkt_event_flg' = False
df = df [df['la_valid_tkt_event_flg'] == True]

# Удаляем строку 'la_valid_tkt_event_flg'
df.drop('la_valid_tkt_event_flg', axis=1, inplace= True)    

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df

,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,minor_cat_name,la_event_type_cat,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,print_flg,Sales site,Sales device
id,,,,,,,,,,,,,,,,,,,,,,
9,97a3efbab1bfcc0a6093,Shlohmo,Shlohmo,Purple,DANCE MUSIC/DANCE,CONCERTS,1,25.0,eTicket,2015-05-01 03:00:00,2015-04-30,NaT,2014-12-12,2015-04-28 07:07:40,2015-04-27,2015-04-30,PST,SAN FRANCISCO,CALIFORNIA,True,www.livenation.com,mobile
11,aacf83f268bd967f98e7,Blake Shelton: Presented By Gildan,Blake Shelton,Chris Janson,COUNTRY,CONCERTS,2,90.0,eTicket,2016-02-21 00:30:00,2016-02-20,2015-12-09,2015-12-11,2015-12-09 15:08:33,2015-12-09,2015-12-09,EST,AUBURN HILLS,MICHIGAN,True,www.ticketmaster.com,desktop
16,0782b2074b13d4bd4756,Lindsey Stirling: The Music Box Tour,Lindsey Stirling,NaN,ROCK/POP,CONCERTS,2,70.0,eTicket,2015-06-10 00:00:00,2015-06-09,2015-03-19,2015-03-20,2015-06-01 20:52:50,2015-06-01,2015-06-05,CST,CLEVELAND,OHIO,True,www.livenation.com,desktop
26,42c5142562a4a9a01798,Godsmack,Godsmack,Within Reason,ALTERNATIVE ROCK,CONCERTS,1,45.0,eTicket,2015-04-18 01:00:00,2015-04-17,2015-02-04,2015-02-06,2015-04-16 22:58:56,2015-04-16,2015-04-16,CST,DALLAS,TEXAS,True,www.livenation.com,desktop
29,e6c30f65b31f7d1c43d2,Florence And The Machine: How Beautiful Tour,Florence and the Machine,Of Monsters and Men,ROCK/POP,CONCERTS,2,40.0,eTicket,2016-06-07 23:30:00,2016-06-07,2016-01-12,2016-01-16,2016-01-16 19:30:08,2016-01-16,2016-01-16,EST,MANSFIELD,MASSACHUSETTS,True,www.livenation.com,desktop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,2c2885f4d6b7c1cf720b,Van Halen,Van Halen,The Kenny Wayne Shepherd Band,ROCK/POP,CONCERTS,2,165.0,TicketFast,2015-10-03 02:30:00,2015-10-02,2015-03-31,2015-04-04,2015-04-10 16:38:41,2015-04-10,2015-04-26,PST,HOLLYWOOD,CALIFORNIA,True,www.ticketmaster.com,desktop
4991,16a410473fc4d3c334d9,Dave Chappelle,Dave Chappelle,NaN,COMEDY,ARTS,2,110.0,eTicket,2015-10-04 23:00:00,2015-10-04,NaT,2015-08-17,2015-08-17 14:04:48,2015-08-17,2015-08-17,EST,ALBANY,NEW YORK,True,www.ticketmaster.com,desktop
4994,5f0e8870d2d08f157042,Josh Groban,Josh Groban,NaN,ADULT CONTEMPORARY,CONCERTS,2,598.0,BWC,2015-09-27 00:00:00,2015-09-26,2015-04-20,2015-04-24,2015-04-20 15:00:07,2015-04-20,2015-08-26,EST,WALLINGFORD,CONNECTICUT,True,www.livenation.com,desktop


## Шаг 7. Переименовываем строки
Считаем, что наш массив данных готов к работе, осталось переименовать строки


In [ ]:
df.rename(columns= {'purch_party_lkup_id': 'Purchase id', 'event_name': 'Event Name'\
                   ,'primary_act_name':'Primary artist', 'secondary_act_name':'Additional artist', 'minor_cat_name' : 'Genre'\
                   ,'la_event_type_cat' : 'Type', 	'tickets_purchased_qty' : 'Tickets purchased'\
                   , 'trans_face_val_amt' : 'Ticket value, $', 	'delivery_type_cd' : 'Delivery type'\
                   ,'event_date_time': 'Event datetime', 	'event_dt' : 'Event date'\
                    , 'presale_dt' : 'Presale date', 	'onsale_dt' : 'Onsale date'\
                    , 'sales_ord_create_dttm' : 'Purchased datetime', 'sales_ord_tran_dt': ''\
                    , 'print_dt' : '', 	'timezn_nm' : 'Concert city timezone', 'venue_city' : 'Venue city'\
                    , 'venue_state' : 'Venue state', 'print_flg': 'Tickets print'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [ ]:
df

,Purchase id,Event Name,Primary artist,Additional artist,Genre,Type,Tickets purchased,"Ticket value, $",Delivery type,Event datetime,Event date,Presale date,Onsale date,Purchased datetime,,,Concert city timezone,Venue city,Venue state,Tickets print,Sales site,Sales device
id,,,,,,,,,,,,,,,,,,,,,,
9,97a3efbab1bfcc0a6093,Shlohmo,Shlohmo,Purple,DANCE MUSIC/DANCE,CONCERTS,1,25.0,eTicket,2015-05-01 03:00:00,2015-04-30,NaT,2014-12-12,2015-04-28 07:07:40,2015-04-27,2015-04-30,PST,SAN FRANCISCO,CALIFORNIA,True,www.livenation.com,mobile
11,aacf83f268bd967f98e7,Blake Shelton: Presented By Gildan,Blake Shelton,Chris Janson,COUNTRY,CONCERTS,2,90.0,eTicket,2016-02-21 00:30:00,2016-02-20,2015-12-09,2015-12-11,2015-12-09 15:08:33,2015-12-09,2015-12-09,EST,AUBURN HILLS,MICHIGAN,True,www.ticketmaster.com,desktop
16,0782b2074b13d4bd4756,Lindsey Stirling: The Music Box Tour,Lindsey Stirling,NaN,ROCK/POP,CONCERTS,2,70.0,eTicket,2015-06-10 00:00:00,2015-06-09,2015-03-19,2015-03-20,2015-06-01 20:52:50,2015-06-01,2015-06-05,CST,CLEVELAND,OHIO,True,www.livenation.com,desktop
26,42c5142562a4a9a01798,Godsmack,Godsmack,Within Reason,ALTERNATIVE ROCK,CONCERTS,1,45.0,eTicket,2015-04-18 01:00:00,2015-04-17,2015-02-04,2015-02-06,2015-04-16 22:58:56,2015-04-16,2015-04-16,CST,DALLAS,TEXAS,True,www.livenation.com,desktop
29,e6c30f65b31f7d1c43d2,Florence And The Machine: How Beautiful Tour,Florence and the Machine,Of Monsters and Men,ROCK/POP,CONCERTS,2,40.0,eTicket,2016-06-07 23:30:00,2016-06-07,2016-01-12,2016-01-16,2016-01-16 19:30:08,2016-01-16,2016-01-16,EST,MANSFIELD,MASSACHUSETTS,True,www.livenation.com,desktop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,2c2885f4d6b7c1cf720b,Van Halen,Van Halen,The Kenny Wayne Shepherd Band,ROCK/POP,CONCERTS,2,165.0,TicketFast,2015-10-03 02:30:00,2015-10-02,2015-03-31,2015-04-04,2015-04-10 16:38:41,2015-04-10,2015-04-26,PST,HOLLYWOOD,CALIFORNIA,True,www.ticketmaster.com,desktop
4991,16a410473fc4d3c334d9,Dave Chappelle,Dave Chappelle,NaN,COMEDY,ARTS,2,110.0,eTicket,2015-10-04 23:00:00,2015-10-04,NaT,2015-08-17,2015-08-17 14:04:48,2015-08-17,2015-08-17,EST,ALBANY,NEW YORK,True,www.ticketmaster.com,desktop
4994,5f0e8870d2d08f157042,Josh Groban,Josh Groban,NaN,ADULT CONTEMPORARY,CONCERTS,2,598.0,BWC,2015-09-27 00:00:00,2015-09-26,2015-04-20,2015-04-24,2015-04-20 15:00:07,2015-04-20,2015-08-26,EST,WALLINGFORD,CONNECTICUT,True,www.livenation.com,desktop


###Вывод
Был проанализирован массив информации о билетах на концерты

Из 46 столбцов осталось всего 22
А из 5000 строк - 2015

---

Часть информации была переформатирована или переконвертирована в другие типы данных.

В результате с данными можно удобнее работать, а сам датасет занимает меньше места. 
